In [ ]:
!pip install nltk pymorphy2

In [ ]:
!pip install pymorphy2


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import pymorphy2

from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Таким образом провожу чтение файла, так как выдает ошибку связанную с кодировкой.
# импортирую библиотеку chardet, которая используется для определения кодировки текстовых файлов
import chardet

#Открытие файла в бинарном режиме
with open('/content/drive/MyDrive/Colab Notebooks/spam.csv', 'rb') as f:
    result = chardet.detect(f.read()) # опеределяю кодировку
    encoding = result['encoding']

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/spam.csv', encoding=encoding)


In [ ]:
df.head(5)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
#Копируем датафрейм
df_clean = df.copy()

#spam = 1, ham = 0. Преобразуем данные в бинарный вид и удаляем не нужные столбцы
df_clean['v1_binary'] = df_clean['v1'].map({'ham': 0, 'spam': 1})
df_clean = df_clean.drop(columns=['v1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'])

df_clean.head(5)

,v2,v1_binary
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


In [ ]:
#Выводим количество уникальных значений в столбце 'toxic' и их доли (частоту) относительно общего числа записей.
# Параметр normalize=True приводит к тому, что вместо абсолютных значений выводятся относительные частоты (проценты).
df_clean.v1_binary.value_counts(normalize=True)

,proportion
v1_binary,
0,0.865937
1,0.134063


In [ ]:
#Установливаем необходимые ресурсы для NLTK
nltk.download('punkt')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer() #легматизатор
# Функция для лемматизации текста
def lemmatize_text(text):
    words = nltk.word_tokenize(text)  # Токенизация
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words] # Лемматизация
    return ' '.join(lemmatized_words) # Объединение в строку

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
df_clean['lemmatized_text'] = df_clean['v2'].apply(lemmatize_text) # Создаем новый легматизированный столбец
df_clean.head(5)

,v2,v1_binary,lemmatized_text
0,"Go until jurong point, crazy.. Available only ...",0,"Go until jurong point , crazy .. Available onl..."
1,Ok lar... Joking wif u oni...,0,Ok lar ... Joking wif u oni ...
2,Free entry in 2 a wkly comp to win FA Cup fina...,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,U dun say so early hor... U c already then say...,0,U dun say so early hor ... U c already then sa...
4,"Nah I don't think he goes to usf, he lives aro...",0,"Nah I do n't think he go to usf , he life arou..."


In [ ]:
train, test = train_test_split(df_clean, test_size = 0.3, shuffle=True) # Разделяем данные на тренировочную и тестовую выборку

In [ ]:
#сбрасываем индексы. Без создания копии
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [ ]:
vectorizer1 = CountVectorizer() #мешок слов

x1 = vectorizer1.fit_transform(train.lemmatized_text) #Применяем метод fit_transform к столбцу 'lemmatized_text' из обучающей выборки.

x1.shape #результат ввиде матрицы. Слов в 2 раза больше чем документов

(3900, 6876)

In [ ]:
vectorizer2 = TfidfVectorizer(min_df=5, max_df=0.4) #tf - idf

x2 = vectorizer2.fit_transform(train.lemmatized_text) #Применяем метод fit_transform к столбцу 'lemmatized_text' из обучающей выборки.
x2_test = vectorizer2.transform(test.lemmatized_text) #Применяем метод fit_transform к столбцу 'lemmatized_text' из тестовой выборки.
# выводим результат ввиде матрицы
x2.shape, x2_test.shape

((3900, 1369), (1672, 1369))

In [ ]:
#извлечение значений целевой переменной для обучения и тестирования модели
y = train.v1_binary.values
y_test = test.v1_binary.values

In [ ]:
x2.shape, y.shape

((3900, 1369), (3900,))

In [ ]:
x2

<3900x1369 sparse matrix of type '<class 'numpy.float64'>'
	with 43195 stored elements in Compressed Sparse Row format>

In [ ]:
x2.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
# Создаем модель случайного леса
rf = RandomForestClassifier(n_estimators=120, random_state=42, max_depth= 30)
rf.fit(x2, y)

preds1 = rf.predict(x2_test)

print(classification_report(y_test, preds1, zero_division=0))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1449
           1       1.00      0.87      0.93       223

    accuracy                           0.98      1672
   macro avg       0.99      0.93      0.96      1672
weighted avg       0.98      0.98      0.98      1672



In [ ]:
  # Создаем модель логистической регрессии
  lr = LogisticRegression(C = 0.5)

  lr.fit(x2, y)
  preds2 = lr.predict(x2_test)

  print(classification_report(y_test, preds2))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      1449
           1       1.00      0.76      0.87       223

    accuracy                           0.97      1672
   macro avg       0.98      0.88      0.92      1672
weighted avg       0.97      0.97      0.97      1672



In [ ]:
# Создаем модель к ближайших соседей
knn = KNeighborsClassifier(n_neighbors=10, metric='cosine')

knn.fit(x2,y)
pred3 = knn.predict(x2_test)

print(classification_report(y_test, pred3))


              precision    recall  f1-score   support

           0       0.97      1.00      0.99      1449
           1       0.98      0.83      0.90       223

    accuracy                           0.97      1672
   macro avg       0.98      0.91      0.94      1672
weighted avg       0.98      0.97      0.97      1672



Вывод:
Случайный лес показывает лучшие результаты по всем метрикам, особенно по полноте класса 1. Это делает его лучшей моделью для данной задачи.

KNN показывает чуть менее высокие результаты,особенно в классе 1, но все равно подходит для практического использования.

Логистическая регрессия имеет схожую общую точность, но менее эффективна при обнаружении спама,чем другие модели. Хотя точность класса 1 высока,его полнота низкая.